# Katrina Inflow Analysis
## Organising the Dataset
We import ```pandas```, ```numpy```, ```csv```, and ```os``` libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import os

We store the path to the ```IRS_migration_data``` repository folder in a string variable

In [2]:
repo_path = os.getcwd()[0:len(os.getcwd())-7]

We also create a new ```tables``` folder in which we will store the tables produced by this script, it will be a subfolder of your ```IRS_migration_data``` repository. If such a folder already exists, a new one will not be created.

In [22]:
results_path = repo_path + 'tables/'
if not os.path.exists(results_path):
    os.makedirs(results_path)

We upload the data on outflows from a csv file. It covers the period 1998-2015.

In [5]:
inflow_df = pd.read_csv(repo_path + 'inflows/inflow.csv')

We print the first 10 lines of the dataframe wew just created to see how it is structured

In [6]:
inflow_df.head()

,year,state_code_dest,county_code_dest,destination,state_code_origin,county_code_origin,origin,state_abbrv,county_name,return_num,exmpt_num,aggr_agi
0,1998-1999,0,0,0,96,0,96000,US,Total Mig - US & For,6916177,13132561,272835496
1,1998-1999,0,0,0,97,0,97000,US,Total Mig - US,6711480,12800546,268111409
2,1998-1999,0,0,0,97,1,97001,US,Total Mig - US Same St,3803650,7221528,143410291
3,1998-1999,0,0,0,97,3,97003,US,Total Mig - US Diff St,2907830,5579018,124701118
4,1998-1999,0,0,0,98,0,98000,US,Total Mig - Foreign,204697,332015,4724087


We store in a numpy array the unique fip codes of destination and origin counties

In [7]:
destinations = inflow_df[(inflow_df['state_code_dest']<=56) & ((inflow_df['state_code_origin']<=56))]
destination_codes = pd.unique(destinations['destination'].values)
origin_codes = np.append(destination_codes, [58000,59000,57009])

We upload the cleaned and restructured datasets for the nine period we will analyse:
    
* before Katrina (1999-2004);
* and after Katrina (2007-2009).

In [8]:
pre_1 = pd.read_csv(repo_path + 'inflows/9900in.csv')
pre_1.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_1.set_index([''], inplace=True)
new_col_names = list(map(int, pre_1.columns.values))
pre_1.columns = new_col_names

pre_2 = pd.read_csv(repo_path + 'inflows/0001in.csv')
pre_2.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_2.set_index([''], inplace=True)
new_col_names = list(map(int, pre_2.columns.values))
pre_2.columns = new_col_names

pre_3 = pd.read_csv(repo_path + 'inflows/0102in.csv')
pre_3.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_3.set_index([''], inplace=True)
new_col_names = list(map(int, pre_3.columns.values))
pre_3.columns = new_col_names

pre_4 = pd.read_csv(repo_path + 'inflows/0203in.csv')
pre_4.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_4.set_index([''], inplace=True)
new_col_names = list(map(int, pre_4.columns.values))
pre_4.columns = new_col_names

pre_5 = pd.read_csv(repo_path + 'inflows/0304in.csv')
pre_5.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_5.set_index([''], inplace=True)
new_col_names = list(map(int, pre_5.columns.values))
pre_5.columns = new_col_names

pre_6 = pd.read_csv(repo_path + 'inflows/0405in.csv')
pre_6.rename(columns={'Unnamed: 0':''}, inplace=True)
pre_6.set_index([''], inplace=True)
new_col_names = list(map(int, pre_6.columns.values))
pre_6.columns = new_col_names

re_1 = pd.read_csv(repo_path + 'inflows/0708in.csv')
re_1.rename(columns={'Unnamed: 0':''}, inplace=True)
re_1.set_index([''], inplace=True)
new_col_names = list(map(int, re_1.columns.values))
re_1.columns = new_col_names

re_2 = pd.read_csv(repo_path + 'inflows/0809in.csv')
re_2.rename(columns={'Unnamed: 0':''}, inplace=True)
re_2.set_index([''], inplace=True)
new_col_names = list(map(int, re_2.columns.values))
re_2.columns = new_col_names

re_3 = pd.read_csv(repo_path + 'inflows/0910in.csv')
re_3.rename(columns={'Unnamed: 0':''}, inplace=True)
re_3.set_index([''], inplace=True)
new_col_names = list(map(int, re_3.columns.values))
re_3.columns = new_col_names

We import a set of csv files that contain the fip codes for different groups of counties we will use in the analysis.

In [10]:
disaster_kat_counties_df = pd.read_csv(repo_path + 'county_groups/disaster_kat_counties.csv', usecols = ['fip_code'])
nearby_kat_counties_df = pd.read_csv(repo_path + 'county_groups/nearby_kat_counties.csv', usecols = ['fip_code'])
distant_kat_counties_df = pd.read_csv(repo_path + 'county_groups/distant_kat_counties.csv', usecols = ['fip_code'])
urban_nc_counties_df = pd.read_csv(repo_path + 'county_groups/urban_nc_counties.csv', usecols = ['fip_code'])

We now convert the dataframes into lists and we add one list with all the counties

In [11]:
disaster_kat_counties = list(disaster_kat_counties_df['fip_code'])
nearby_kat_counties = list(nearby_kat_counties_df['fip_code'])
distant_kat_counties = list(distant_kat_counties_df['fip_code'])
urban_nc_counties = list(urban_nc_counties_df['fip_code'])
all_nc_counties = disaster_kat_counties + nearby_kat_counties + distant_kat_counties

Finally, using list comprehension, we divide all the groups we have defined so far into urban and rural areas by looking at their 2010 Census population. If the proportion living in rural areas is equal or above 50% we classify the county as rural otherwise as urban.

In [12]:
disaster_kat_urban_counties = [x for x in disaster_kat_counties if x in urban_nc_counties]
nearby_kat_urban_counties = [x for x in nearby_kat_counties if x in urban_nc_counties]
distant_kat_urban_counties = [x for x in distant_kat_counties if x in urban_nc_counties]

We now summarize the number of counties in each group:

In [13]:
print('There are', len(disaster_kat_counties), 'disaster counties, of which', 
      len(disaster_kat_urban_counties), 'are urban.' )

print('There are', len(nearby_kat_counties), 'nearby counties, of which', 
      len(nearby_kat_urban_counties), 'are urban.' )

print('There are', len(distant_kat_counties), 'distant counties, of which', 
      len(distant_kat_urban_counties), 'are urban.' )

print('There is a total of', len(all_nc_counties), 'counties, of which', 
      len(urban_nc_counties), 'are urban and the remaining',
      len(all_nc_counties) - len(urban_nc_counties), 'are rural.' )

There are 36 disaster counties, of which 31 are urban.
There are 124 nearby counties, of which 63 are urban.
There are 2952 distant counties, of which 1153 are urban.
There is a total of 3112 counties, of which 1247 are urban and the remaining 1865 are rural.


## Ties Analysis
We create six dataframes, one for each year considered, where we have the ties connecting each county to the others. Here a tie is defined as the presence of a flow of any size between two counties. The final result is a matrix for each period whose rows and columns are all the counties in the dataset and where a 1 indicates the presence of a tie between the two counties and a 0 its absence. We consider a tie to exist if a positive flow was recorded at least in one of the years composing the before and after periods.

In [14]:
pre_1_ties_kat = pre_1.drop([58000,59000,57009],axis=1).where(pre_1==0,1)
pre_2_ties_kat = pre_2.drop([58000,59000,57009],axis=1).where(pre_2==0,1)
pre_3_ties_kat = pre_3.drop([58000,59000,57009],axis=1).where(pre_3==0,1)
pre_4_ties_kat = pre_4.drop([58000,59000,57009],axis=1).where(pre_4==0,1)
pre_5_ties_kat = pre_5.drop([58000,59000,57009],axis=1).where(pre_5==0,1)
pre_6_ties_kat = pre_6.drop([58000,59000,57009],axis=1).where(pre_6==0,1)

re_1_ties_kat = re_1.drop([58000,59000,57009],axis=1).where(re_1==0,1)
re_2_ties_kat = re_2.drop([58000,59000,57009],axis=1).where(re_2==0,1)
re_3_ties_kat = re_3.drop([58000,59000,57009],axis=1).where(re_3==0,1)

In [15]:
pre_ties_kat = (pre_1_ties_kat + pre_2_ties_kat + pre_3_ties_kat + pre_4_ties_kat + pre_5_ties_kat + pre_6_ties_kat)
pre_ties_kat = pre_ties_kat.where(pre_ties_kat==0,1)

re_ties_kat = (re_1_ties_kat + re_2_ties_kat + re_3_ties_kat)
re_ties_kat = re_ties_kat.where(re_ties_kat==0,1)

The final result is a matrix for each period whose rows and columns are all the counties in the dataset and where a 1 indicates the presence of a tie between the two counties and a 0 its absence. We consider a tie to exist if a positive flow was recorded at least in one of the three years composing the before and after periods.

In [16]:
ties_kat = re_ties_kat - pre_ties_kat
uties_pre_kat = ties_kat.where(ties_kat==-1,0)*-1
uties_re_kat = ties_kat.where(ties_kat==1,0)

In [17]:
for i in uties_pre_kat.index:
    uties_pre_kat.loc[i, i] = 0
for i in uties_re_kat.index:
    uties_re_kat.loc[i, i] = 0

We set up the table headers

In [18]:
counties_groups_kat = ['All','Disaster Affected','Nearby','Distant',
                   'All (Urban)', 'Disaster Affected (Urban)', 'Nearby (Urban)','Distant (Urban)']
periods = ['Pre-Disaster','Recovery','% Change']

We initialise the table

In [19]:
ties_df_kat = pd.DataFrame(0, index=counties_groups_kat, columns=periods)

Finally we fill it

In [20]:
ties_df_kat.loc['All',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties, 
                                                all_nc_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['All',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties, 
                                           all_nc_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Disaster Affected',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                disaster_kat_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Disaster Affected',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           disaster_kat_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Nearby',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                nearby_kat_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Nearby',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           nearby_kat_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Distant',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                distant_kat_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Distant',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           distant_kat_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['All (Urban)',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                urban_nc_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['All (Urban)',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           urban_nc_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                disaster_kat_urban_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Disaster Affected (Urban)',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           disaster_kat_urban_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Nearby (Urban)',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                nearby_kat_urban_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Nearby (Urban)',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           nearby_kat_urban_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc['Distant (Urban)',
            'Pre-Disaster'] = uties_pre_kat.loc[disaster_kat_counties,
                                                distant_kat_urban_counties].sum(axis=1).sum(axis=0)
ties_df_kat.loc['Distant (Urban)',
            'Recovery'] = uties_re_kat.loc[disaster_kat_counties,
                                           distant_kat_urban_counties].sum(axis=1).sum(axis=0)

ties_df_kat.loc[:,'% Change'] = (ties_df_kat.loc[:,'Recovery'] - ties_df_kat.loc[:,'Pre-Disaster'])/ties_df_kat.loc[:,'Pre-Disaster']*100

ties_df_kat.loc[:,'% Change'] = ties_df_kat.loc[:,'% Change'].round(decimals=1)
ties_df_kat


,Pre-Disaster,Recovery,% Change
All,457.0,444.0,-2.8
Disaster Affected,46.0,30.0,-34.8
Nearby,72.0,98.0,36.1
Distant,339.0,316.0,-6.8
All (Urban),384.0,379.0,-1.3
Disaster Affected (Urban),44.0,27.0,-38.6
Nearby (Urban),50.0,73.0,46.0
Distant (Urban),290.0,279.0,-3.8


To export the table to a csv file, uncomment the following line

In [23]:
#ties_df_kat.to_csv(results_path + 'inties_table_kat.csv')

## Flows Analysis
We create to dataframes with the same structure as the ones with the ties but containing average flows for the two periods respectively

In [24]:
pre_avg_kat = (pre_1 + pre_2 + pre_3 + pre_4 + pre_5 + pre_6)/6
re_avg_kat = (re_1 + re_2 + re_3)/3

pre_avg_kat = pre_avg_kat.round(decimals=0)
re_avg_kat = re_avg_kat.round(decimals=0)

pre_flows_kat = pre_avg_kat
re_flows_kat = re_avg_kat

We remove flows in the main diagonal as these represent household that remained in the same counties and are thus not interesting in our migration analysis 

In [25]:
for i in pre_flows_kat.index:
    pre_flows_kat.loc[i, i] = 0
for i in re_flows_kat.index:
    re_flows_kat.loc[i, i] = 0

We initialise the flows table as before

In [26]:
flows_df_kat = pd.DataFrame(0, index=counties_groups_kat, columns=periods)

and now we fill it

In [37]:
flows_df_kat.loc['All',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties,
                                              all_nc_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['All',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties,
                                         all_nc_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Disaster Affected',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              disaster_kat_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Disaster Affected',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         disaster_kat_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Nearby',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              nearby_kat_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['Nearby',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         nearby_kat_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Distant',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              distant_kat_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['Distant',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         distant_kat_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['All (Urban)',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              urban_nc_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['All (Urban)',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         urban_nc_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Disaster Affected (Urban)',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              disaster_kat_urban_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['Disaster Affected (Urban)',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         disaster_kat_urban_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Nearby (Urban)',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              nearby_kat_urban_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['Nearby (Urban)',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         nearby_kat_urban_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc['Distant (Urban)',
            'Pre-Disaster'] = pre_avg_kat.loc[disaster_kat_counties, 
                                              distant_kat_urban_counties].sum(axis=1).sum(axis=0)
flows_df_kat.loc['Distant (Urban)',
            'Recovery'] = re_avg_kat.loc[disaster_kat_counties, 
                                         distant_kat_urban_counties].sum(axis=1).sum(axis=0)

flows_df_kat.loc[:,'% Change'] = (flows_df_kat.loc[:,'Recovery'] - flows_df_kat.loc[:,'Pre-Disaster'])/flows_df_kat.loc[:,'Pre-Disaster']*100

flows_df_kat.loc[:,'% Change'] = flows_df_kat.loc[:,'% Change'].round(decimals=1)

flows_df_kat

,Pre-Disaster,Recovery,% Change
All,121259.0,144847.0,19.5
Disaster Affected,49969.0,54030.0,8.1
Nearby,23595.0,30868.0,30.8
Distant,47695.0,59949.0,25.7
All (Urban),114080.0,136567.0,19.7
Disaster Affected (Urban),49624.0,53606.0,8.0
Nearby (Urban),19421.0,26004.0,33.9
Distant (Urban),45035.0,56957.0,26.5


To export the table to a csv file, uncomment the following line

In [38]:
#flows_df_kat.to_csv(results_path + 'inflows_table_katrina.csv')